In [57]:
import re
import nltk
from nltk.corpus import stopwords
import math as m
import operator
from fractions import Fraction
from nltk.stem.porter import PorterStemmer

In [58]:
#Question term generator
def Question_term(question):
    stop = set(stopwords.words('english'))
    pattern = r'[^A-Za-z0-9\s]+'
    question = re.sub(pattern,'', question)
    token = nltk.word_tokenize(question)
    cach = []
    for word in token:
        if word not in stop:
            cach.append(word)
    pos_tags = nltk.pos_tag(cach)

    query_term = {}

    for item in pos_tags:
        if item[0] != 'Which' and item[0] != 'What' and item[0] != 'Where' and item[0] != 'When' and item[0] != 'Who' and item[0] != 'Whose' and item[0] != 'How' and  item[0] not in query_term:
            query_term[item[0]] = 1
        elif item[0] != 'Which' and item[0] != 'What' and item[0] != 'Where' and item[0] != 'When' and item[0] != 'Who' and item[0] != 'Whose' and item[0] != 'How' and item[0] in query_term:
            query_term[item[0]] = query_term[item[0]] + 1

    return query_term

In [59]:
#find the document which contains at least one of the keywords
def DocumentRetrieve(path, keywords):
    file = open(path)
    flag = False
    for row in file:
        row = row.replace('\n', '')
        row = row.replace('\r', '')
        row = row.split()
        
        for word in row:
            if keywords.has_key(word):
                flag = True
                break
                
    if flag:
        return path
    else:
        return ''

In [60]:
#score the file that contains keywords
def TF_IDF(keywords, All_relevant_Document, totalNum):
    qtf = keywords
    df = Document_Frequency(keywords, All_relevant_Document)
    total_tf = {}
    
    for document in All_relevant_Document:
        tf = Term_Frequency(keywords, document)
        if tf != {}:
            total_tf[document] = tf
            
    TFIDF = {}
    for key,val in total_tf.items():
        max_term = maximumTerm(key)
        term = val
        score = 0
        for eachterm, value in term.iteritems():
            TF = 0.5 + (0.5 * value / max_term)
            IDF = m.log(Fraction(totalNum, df[eachterm]))
            
            tf_idf = TF * IDF
            score = score + tf_idf
            
        if score != 0:
            TFIDF[key] = score
            
        return sorted(TFIDF.iteritems(), key = operator.itemgetter(1), reverse = True)

In [61]:
#compute the maximum term frequency of specific document
def maximumTerm(Document):
    file = open(Document)
    Maximum_term = {}
    stop = set(stopwords.words('English'))
    for row in file:
        row = row.replace('\n','')
        words = row.split(' ')
        for word in words:
            if word not in stop and word.lower() not in stop:
                if not Maximum_term.has_key(word):
                    Maximum_term[word] = 1
                else:
                    Maximum_term[word] = Maximum_term[word] + 1
                    
    return max(Maximum_term.iteritems(), key = operator.itemgetter(1))[1]

In [62]:
#compute document_frequency
def Document_Frequency(queryterms, All_relevant_Document):
    df = dict.fromkeys(queryterms.keys(), 0)
    for term in df:
        for document in All_relevant_Document:
            flag = False
            file = open(document)
            for row in file:
                row = row.replace('\n', '')
                words = row.split(' ')
                for word in words:
                    if word == term:
                        df[term] = df[term] + 1
                        flag = True
                        break
                if flag == True:
                    break
    return df

In [63]:
#compute term_frequency
def Term_Frequency(queryterms, Document):
    tf = {}
    file = open(Document)
    for row in file:
        row = row.replace('\n','')
        words = row.split(' ')
        for word in words:
            if queryterms.has_key(word) and not tf.has_key(word):
                tf[word] = 1
            elif queryterms.has_key(word) and tf.has_key(word):
                tf[word] = tf[word] + 1
    
    return tf

In [64]:
#find Num Documents with high score
def Document_subset(Document, Num):
    result = []
    for i in range(0, Num):
        result.append(Document[i][0])
    
    return result

In [65]:
def All_Document(key_words):
    All_relevant_Document = []
    for year in range(3,5):
        for month in range(1,13):
            for day in range(1,32):
                if month < 10 and day < 10:
                    file_path ='201' + str(year) + '/' + '201' + str(year) + '-0' + str(month) + '-0' + str(day) + '.txt'
                elif month < 10 and day >= 10:
                    file_path ='201' + str(year) + '/' + '201' + str(year) + '-0' + str(month) + '-' + str(day) + '.txt'
                elif month > 10 and day < 10:
                    file_path ='201' + str(year) + '/' + '201' + str(year) + '-' + str(month) + '-0' + str(day) + '.txt'
                else:
                    file_path ='201' + str(year) + '/' + '201' + str(year) + '-' + str(month) + '-' + str(day) + '.txt'
                    document = ''
                try:
                    document = DocumentRetrieve(file_path,key_words)
                except Exception:
                    continue
                if document != '':
                    All_relevant_Document.append(document)
    
    return All_relevant_Document

In [66]:
question = 'What is the unemployment rate?'
queryterms = Question_term(question)
print(queryterms)

{'unemployment': 1, 'rate': 1}


In [67]:
All_relevant_Document = All_Document(queryterms)
ans = TF_IDF(queryterms,All_relevant_Document,len(All_relevant_Document))